## Deliverable 2. Create a Customer Travel Destinations Map.

In [68]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,-46.2817,169.8464,49.68,95,100,5.86
1,1,San Carlos De Bariloche,AR,-41.1456,-71.3082,56.19,51,4,7.29
2,2,Sibu,MY,2.3000,111.8167,72.59,96,92,2.01
3,3,Vaini,TO,-21.2000,-175.2000,75.36,88,40,9.22
4,4,Shorewood,US,41.5200,-88.2017,68.76,40,1,11.50


In [30]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Sibu,MY,2.3000,111.8167,72.59,96,92,2.01
3,3,Vaini,TO,-21.2000,-175.2000,75.36,88,40,9.22
4,4,Shorewood,US,41.5200,-88.2017,68.76,40,1,11.50
7,7,Saint-Philippe,RE,-21.3585,55.7679,71.15,80,46,14.70
9,9,Mahebourg,MU,-20.4081,57.7000,73.26,75,47,16.78
15,15,Yulara,AU,-25.2406,130.9889,67.84,28,6,4.61
16,16,Bambous Virieux,MU,-20.3428,57.7575,73.65,78,40,12.66
19,19,Namibe,AO,-15.1961,12.1522,65.39,90,37,5.26
24,24,Vao,NC,-22.6667,167.4833,72.72,88,4,11.52
25,25,Dauphin,CA,51.1494,-100.0502,68.70,30,4,9.06


In [32]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       227
City          227
Country       226
Lat           227
Lng           227
Max Temp      227
Humidity      227
Cloudiness    227
Wind Speed    227
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Sibu,MY,2.3000,111.8167,72.59,96,92,2.01
3,3,Vaini,TO,-21.2000,-175.2000,75.36,88,40,9.22
4,4,Shorewood,US,41.5200,-88.2017,68.76,40,1,11.50
7,7,Saint-Philippe,RE,-21.3585,55.7679,71.15,80,46,14.70
9,9,Mahebourg,MU,-20.4081,57.7000,73.26,75,47,16.78
15,15,Yulara,AU,-25.2406,130.9889,67.84,28,6,4.61
16,16,Bambous Virieux,MU,-20.3428,57.7575,73.65,78,40,12.66
19,19,Namibe,AO,-15.1961,12.1522,65.39,90,37,5.26
24,24,Vao,NC,-22.6667,167.4833,72.72,88,4,11.52
25,25,Dauphin,CA,51.1494,-100.0502,68.70,30,4,9.06


In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Sibu,MY,72.59,2.3000,111.8167,
3,Vaini,TO,75.36,-21.2000,-175.2000,
4,Shorewood,US,68.76,41.5200,-88.2017,
7,Saint-Philippe,RE,71.15,-21.3585,55.7679,
9,Mahebourg,MU,73.26,-20.4081,57.7000,
15,Yulara,AU,67.84,-25.2406,130.9889,
16,Bambous Virieux,MU,73.65,-20.3428,57.7575,
19,Namibe,AO,65.39,-15.1961,12.1522,
24,Vao,NC,72.72,-22.6667,167.4833,
25,Dauphin,CA,68.70,51.1494,-100.0502,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()
clean_hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Sibu,MY,72.59,2.3000,111.8167,RH HOTEL (4 Star)
3,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
4,Shorewood,US,68.76,41.5200,-88.2017,Wingate by Wyndham Joliet
7,Saint-Philippe,RE,71.15,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Mahebourg,MU,73.26,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Yulara,AU,67.84,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
16,Bambous Virieux,MU,73.65,-20.3428,57.7575,Casa Tia Villa
19,Namibe,AO,65.39,-15.1961,12.1522,Hotel Chik Chik Namibe
24,Vao,NC,72.72,-22.6667,167.4833,Hotel Kou-Bugny
25,Dauphin,CA,68.70,51.1494,-100.0502,Canway Inn & Suites


In [53]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [69]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))